# Recommender System Project
In this project first I have deployed the 2 methods described in the course

and finally a hybrid score is defined to get better recommendations

**nateghiiman@gmail.com**

In [2]:
import pandas as pd

# Read Data
import from CSV

In [12]:
movies_df=pd.read_csv("C:/python/Jadi_ML/01 Iman/movies.csv")
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [13]:
ratings_df=pd.read_csv("C:/python/Jadi_ML/01 Iman/ratings.csv")
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


# Data Cleaning
Using Regex we split the **title** column into two columns: **title** and **year**

In [14]:
movies_df['year'] = movies_df.title.str.extract('(\(\d\d\d\d\))',expand=False)
#Removing the parentheses
movies_df['year'] = movies_df.year.str.extract('(\d\d\d\d)',expand=False)
#Removing the years from the 'title' column
movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')
#Applying the strip function to get rid of any ending whitespace characters that may have appeared
movies_df['title'] = movies_df['title'].apply(lambda x: x.strip())
movies_df.head()

C:\Users\nateg\AppData\Local\Temp\ipykernel_22444\2552882948.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')


,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


Then Split the **genres** and making a python list of genres instead of single string seperated by "|"

In [15]:
#Every genre is separated by a | so we simply have to call the split function on |
movies_df['genres'] = movies_df.genres.str.split('|')
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995
2,3,Grumpier Old Men,"[Comedy, Romance]",1995
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995
4,5,Father of the Bride Part II,[Comedy],1995


we create a (0/1) column for each genre. The intersection of movie and it's related genres are set to 1 the others 0

In [35]:
#Copying the movie dataframe into a new one since we won't need to use the genre information in our first case.
moviesWithGenres_df = movies_df.copy()

#For every row in the dataframe, iterate through the list of genres and place a 1 into the corresponding column
for index, row in movies_df.iterrows():
    for genre in row['genres']:
        moviesWithGenres_df.at[index, genre] = 1
#Filling in the NaN values with 0 to show that a movie doesn't have that column's genre
moviesWithGenres_df = moviesWithGenres_df.fillna(0)
moviesWithGenres_df.head()

,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men,"[Comedy, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II,[Comedy],1995,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


We dont need timestamp so we remove it

In [16]:
#Drop removes a specified row or column from a dataframe
ratings_df = ratings_df.drop('timestamp', 1)
ratings_df.head()

C:\Users\nateg\AppData\Local\Temp\ipykernel_22444\3391429438.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  ratings_df = ratings_df.drop('timestamp', 1)


,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


# Content Based Recommendor System
As an input I write my own favorite movies

In [24]:
userInput = [
            {'title':'Matrix, The', 'rating':5},
            {'title':'Prestige, The', 'rating':4},
            {'title':'Deadpool 2', 'rating':2},
            {'title':"Pulp Fiction", 'rating':3},
            {'title':'Annihilation', 'rating':3.5}
         ] 
inputMovies = pd.DataFrame(userInput)
inputMovies
#2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller
#48780,"Prestige, The (2006)",Drama|Mystery|Sci-Fi|Thriller
#187593,Deadpool 2 (2018),Action|Comedy|Sci-Fi
#185135,Sherlock - A Study in Pink (2010),Crime
#182715,Annihilation (2018),Adventure|Mystery|Sci-Fi|Thriller
#173145,War for the Planet of the Apes (2017),Action|Adventure|Drama|Sci-Fi
#177185,Maz Jobrani: Immigrant (2017),Comedy
#176601,Black Mirror,(no genres listed)
#172591,The Godfather Trilogy: 1972-1990 (1992),(no genres listed)


,title,rating
0,"Matrix, The",5.0
1,"Prestige, The",4.0
2,Deadpool 2,2.0
3,Pulp Fiction,3.0
4,Annihilation,3.5


select rows from movies dataframe which I have chosen

In [25]:
#Filtering out the movies by title
inputId = movies_df[movies_df['title'].isin(inputMovies['title'].tolist())]
inputId

,movieId,title,genres,year
257,296,Pulp Fiction,"[Comedy, Crime, Drama, Thriller]",1994
1939,2571,"Matrix, The","[Action, Sci-Fi, Thriller]",1999
6331,48780,"Prestige, The","[Drama, Mystery, Sci-Fi, Thriller]",2006
9668,182715,Annihilation,"[Adventure, Mystery, Sci-Fi, Thriller]",2018
9709,187593,Deadpool 2,"[Action, Comedy, Sci-Fi]",2018


Add the rating to the table

In [26]:
#Then merging it so we can get the movieId. It's implicitly merging it by title.
inputMovies = pd.merge(inputId, inputMovies)
inputMovies

,movieId,title,genres,year,rating
0,296,Pulp Fiction,"[Comedy, Crime, Drama, Thriller]",1994,3.0
1,2571,"Matrix, The","[Action, Sci-Fi, Thriller]",1999,5.0
2,48780,"Prestige, The","[Drama, Mystery, Sci-Fi, Thriller]",2006,4.0
3,182715,Annihilation,"[Adventure, Mystery, Sci-Fi, Thriller]",2018,3.5
4,187593,Deadpool 2,"[Action, Comedy, Sci-Fi]",2018,2.0


Again we drop what is unnecessary

In [29]:
#Dropping information we won't use from the input dataframe
inputMovies = inputMovies.drop('genres', 1).drop('year', 1)
inputMovies

,movieId,title,rating
0,296,Pulp Fiction,3.0
1,2571,"Matrix, The",5.0
2,48780,"Prestige, The",4.0
3,182715,Annihilation,3.5
4,187593,Deadpool 2,2.0


nesxt we filter the cleaned movies data frame to only show the movies I had added as input (favorites)

In [36]:
userMovies = moviesWithGenres_df[moviesWithGenres_df['movieId'].isin(inputMovies['movieId'].tolist())]
userMovies

,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
257,296,Pulp Fiction,"[Comedy, Crime, Drama, Thriller]",1994,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1939,2571,"Matrix, The","[Action, Sci-Fi, Thriller]",1999,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6331,48780,"Prestige, The","[Drama, Mystery, Sci-Fi, Thriller]",2006,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9668,182715,Annihilation,"[Adventure, Mystery, Sci-Fi, Thriller]",2018,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9709,187593,Deadpool 2,"[Action, Comedy, Sci-Fi]",2018,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


we create a matrix to perform algebratic operations by droping some colums so we have only genres column (0/1)

In [37]:
#Resetting the index to avoid future issues
userMovies = userMovies.reset_index(drop=True)
#Dropping unnecessary issues due to save memory and to avoid issues
userGenreTable = userMovies.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)
userGenreTable

C:\Users\nateg\AppData\Local\Temp\ipykernel_22444\2641803640.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  userGenreTable = userMovies.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)
C:\Users\nateg\AppData\Local\Temp\ipykernel_22444\2641803640.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  userGenreTable = userMovies.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)
C:\Users\nateg\AppData\Local\Temp\ipykernel_22444\2641803640.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  userGenreTable = userMovies.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)
C:\Users\nateg\AppData\Local\Temp\ipykernel_22444\2641803640.py:4: FutureWarning: In a future version o

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Next we multiply the above Matrix with the corresponding scores stores in rating column of inputMovies

In [38]:
inputMovies['rating']

0    3.0
1    5.0
2    4.0
3    3.5
4    2.0
Name: rating, dtype: float64

each row of the first matix is multiplied by the corresponding column of the second matrix

so we first transpose the 1st matrix to make the 2 matrixes align

In [43]:
#Dot produt to get weights
userProfile = userGenreTable.transpose().dot(inputMovies['rating'])
#The user profile
userProfile

Adventure              3.5
Animation              0.0
Children               0.0
Comedy                 5.0
Fantasy                0.0
Romance                0.0
Drama                  7.0
Action                 7.0
Crime                  3.0
Thriller              15.5
Horror                 0.0
Mystery                7.5
Sci-Fi                14.5
War                    0.0
Musical                0.0
Documentary            0.0
IMAX                   0.0
Western                0.0
Film-Noir              0.0
(no genres listed)     0.0
dtype: float64

now we start to make a list of **ALL** the movies from the cleaned movies dataframe

we drop unnecessary colums so a matrix like user genres table is obtained

In [44]:
#Now let's get the genres of every movie in our original dataframe
genreTable = moviesWithGenres_df.set_index(moviesWithGenres_df['movieId'])
#And drop the unnecessary information
genreTable = genreTable.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)
genreTable.head()

C:\Users\nateg\AppData\Local\Temp\ipykernel_22444\789887408.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  genreTable = genreTable.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)
C:\Users\nateg\AppData\Local\Temp\ipykernel_22444\789887408.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  genreTable = genreTable.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)
C:\Users\nateg\AppData\Local\Temp\ipykernel_22444\789887408.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  genreTable = genreTable.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)
C:\Users\nateg\AppData\Local\Temp\ipykernel_22444\789887408.py:4: FutureWarning: In a future version of pandas all arg

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
movieId,,,,,,,,,,,,,,,,,,,,
1,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [45]:
genreTable.shape

(9742, 20)

Now we calculate each movie's score based on the user profile

In [146]:
#Multiply the genres by the weights and then take the weighted average
CBrecommendationTable = ((genreTable*userProfile).sum(axis=1))/(userProfile.sum())
CBrecommendationTable.head()

movieId
1    0.134921
2    0.055556
3    0.079365
4    0.190476
5    0.079365
dtype: float64

sort the table desendind so the first item is the most relevant movie

In [147]:
#Sort our recommendations in descending order
CBrecommendationTable = CBrecommendationTable.sort_values(ascending=False)
#Just a peek at the values
CBrecommendationTable.head()
type(CBrecommendationTable)

pandas.core.series.Series

In [149]:
### first change the type from series to dataframe so we can merge easily
CBrecommendationTable_df=pd.DataFrame(data = [CBrecommendationTable.values] , columns = CBrecommendationTable.index).transpose()
CBrecommendationTable_df = CBrecommendationTable_df.set_axis(['CB Score'], axis=1, inplace=False)
type(CBrecommendationTable_df)

C:\Users\nateg\AppData\Local\Temp\ipykernel_22444\3922080626.py:3: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  CBrecommendationTable_df = CBrecommendationTable_df.set_axis(['CB Score'], axis=1, inplace=False)


pandas.core.frame.DataFrame

And finally showing the suggestions!

but first change the type from series to dataframe so we can merge easily

In [154]:
#The final recommendation table (this is another way mentioned in course)
##cbresultdf=movies_df.loc[movies_df['movieId'].isin(CBrecommendationTable_df.head(20).keys())]

#-------------------------------------------------

Recommendation_df=CBrecommendationTable_df.merge(movies_df,left_on="movieId",right_on="movieId" )
Recommendation_df=Recommendation_df.sort_values(by="CB Score",ascending=False)
Recommendation_df.head(10)


,movieId,CB Score,title,genres,year
0,79132,0.865079,Inception,"[Action, Crime, Drama, Mystery, Sci-Fi, Thrill...",2010
2,198,0.865079,Strange Days,"[Action, Crime, Drama, Mystery, Sci-Fi, Thriller]",1995
1,26701,0.865079,Patlabor: The Movie (Kidô keisatsu patorebâ: T...,"[Action, Animation, Crime, Drama, Film-Noir, M...",1989
3,6902,0.841270,Interstate 60,"[Adventure, Comedy, Drama, Fantasy, Mystery, S...",2002
4,85414,0.817460,Source Code,"[Action, Drama, Mystery, Sci-Fi, Thriller]",2011
5,60684,0.817460,Watchmen,"[Action, Drama, Mystery, Sci-Fi, Thriller, IMAX]",2009
6,43932,0.817460,Pulse,"[Action, Drama, Fantasy, Horror, Mystery, Sci-...",2006
7,67197,0.817460,Knowing,"[Action, Drama, Mystery, Sci-Fi, Thriller]",2009
8,81132,0.769841,Rubber,"[Action, Adventure, Comedy, Crime, Drama, Film...",2010
9,36509,0.761905,"Cave, The","[Action, Adventure, Horror, Mystery, Sci-Fi, T...",2005


The result seems satisfactory as I have watche some of these, and they are also my favorites


as a disadvatage this method Doesn't take into account what others think of the item, so low quality item recommendations might happen, Extracting data is not always intuitive, Determining what characteristics of the item the user dislikes or likes is not always obvious

so we move on to try out another method:

# Collaborative Recommender System

In [293]:
ratings_df.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [294]:
inputMovies

,movieId,title,rating
0,296,Pulp Fiction,3.0
1,2571,"Matrix, The",5.0
2,48780,"Prestige, The",4.0
3,182715,Annihilation,3.5
4,187593,Deadpool 2,2.0


We form a subset of users who have watched what I have selected in inputMovies

In [295]:
userSubset = ratings_df[ratings_df['movieId'].isin(inputMovies['movieId'].tolist())]
userSubset.head()

,userId,movieId,rating
16,1,296,3.0
166,1,2571,5.0
320,4,296,1.0
442,4,2571,1.0
533,5,296,5.0


Group by userID

In [296]:
userSubsetGroup = userSubset.groupby(['userId'])
userSubsetGroup

we can assess data of each user:

In [297]:
userSubsetGroup.get_group(555)

,userId,movieId,rating
85325,555,296,4.0
85682,555,2571,5.0


In [298]:
userSubsetGroup = sorted(userSubsetGroup,  key=lambda x: len(x[1]), reverse=True)

C:\Users\nateg\AppData\Local\Temp\ipykernel_22444\1827451528.py:1: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  userSubsetGroup = sorted(userSubsetGroup,  key=lambda x: len(x[1]), reverse=True)


let's check the first 3 users

In [299]:
userSubsetGroup[0:3]

[(62,
        userId  movieId  rating
  8797      62      296     4.5
  8835      62     2571     5.0
  8945      62    48780     5.0
  9150      62   182715     4.0
  9155      62   187593     4.0),
 (249,
         userId  movieId  rating
  36398     249      296     4.0
  36556     249     2571     5.0
  36878     249    48780     3.5
  37413     249   182715     4.0
  37419     249   187593     5.0),
 (233,
         userId  movieId  rating
  34765     233      296     4.0
  34799     233     2571     4.5
  34850     233    48780     3.0
  34909     233   187593     2.5)]

![alt text](https://wikimedia.org/api/rest_v1/media/math/render/svg/bd1ccc2979b0fd1c1aec96e386f686ae874f9ec0 "Pearson Correlation")

in order to reduce the amount of calculation we only use the top most similar ratings:

In [300]:
userSubsetGroup = userSubsetGroup[0:100]
len(userSubsetGroup)

100

In [301]:
import scipy

pearsonCorrelationDict = {}
for name, group in userSubsetGroup:
    #Let's start by sorting the input and current user group so the values aren't mixed up later on
    group = group.sort_values(by='movieId')
    inputMovies = inputMovies.sort_values(by='movieId')
    #Get the N for the formula
    nRatings = len(group)
    #Get the review scores for the movies that they both have in common
    temp_df = inputMovies[inputMovies['movieId'].isin(group['movieId'].tolist())]
    #And then store them in a temporary buffer variable in a list format to facilitate future calculations
    tempRatingList = temp_df['rating'].tolist()
    #Let's also put the current user group reviews in a list format
    tempGroupList = group['rating'].tolist()
    #Now let's calculate the pearson correlation between two users, so called, x and y
    pearsonCorrelationDict[name] = (scipy.stats.pearsonr(tempRatingList, tempGroupList).statistic)

pearsonCorrelationDict

C:\Program Files\Python38\lib\site-packages\scipy\stats\_stats_py.py:4427: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))


{62: 0.7826237921249264,
 249: -0.08333333333333333,
 233: 0.7071067811865476,
 305: -0.25819888974716115,
 380: 0.6137949055234262,
 15: -2.424259902380233e-18,
 18: 0.8660254037844386,
 28: -0.15554275420956373,
 41: -0.9819805060619655,
 50: -0.8660254037844387,
 63: -0.8660254037844387,
 68: 0.9933992677987827,
 103: -0.9999999999999998,
 105: -0.8660254037844385,
 122: nan,
 123: nan,
 131: -0.9999999999999998,
 132: 0.4999999999999999,
 166: -0.2401922307076307,
 177: -0.9607689228305228,
 204: -2.424259902380233e-18,
 210: 0.9522165814091075,
 211: 0.4999999999999999,
 219: 0.9999999999999998,
 226: -0.8660254037844386,
 232: -0.8660254037844386,
 239: -5.465852201897649e-18,
 241: -0.4999999999999999,
 247: -0.4999999999999999,
 254: -5.465852201897649e-18,
 261: -0.9999999999999998,
 274: -0.6546536707079771,
 279: -0.8660254037844386,
 282: 0.9999999999999998,
 288: -0.9607689228305227,
 298: -0.1889822365046136,
 309: 0.8660254037844385,
 318: -0.6933752452815365,
 332: 0.65

In [302]:
pearsonDF = pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index')
pearsonDF.columns = ['similarityIndex']
pearsonDF['userId'] = pearsonDF.index
pearsonDF.index = range(len(pearsonDF))
pearsonDF.head()

,similarityIndex,userId
0,0.782624,62
1,-0.083333,249
2,0.707107,233
3,-0.258199,305
4,0.613795,380


In [303]:
topUsers=pearsonDF.sort_values(by='similarityIndex', ascending=False)[0:50]
topUsers.head(20)

,similarityIndex,userId
83,1.000000,78
93,1.000000,137
76,1.000000,57
69,1.000000,21
67,1.000000,16
85,1.000000,91
91,1.000000,125
64,1.000000,1
81,1.000000,72
92,1.000000,135


In [304]:
ratings_df

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
...,...,...,...
100831,610,166534,4.0
100832,610,168248,5.0
100833,610,168250,5.0
100834,610,168252,5.0


In [305]:
topUsersRating=topUsers.merge(ratings_df, left_on='userId', right_on='userId', how='inner')
topUsersRating.head()

,similarityIndex,userId,movieId,rating
0,1.0,78,1,4.0
1,1.0,78,20,3.0
2,1.0,78,32,3.5
3,1.0,78,150,4.0
4,1.0,78,165,3.0


In [306]:
#Multiplies the similarity by the user's ratings
topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['rating']
topUsersRating

,similarityIndex,userId,movieId,rating,weightedRating
0,1.0,78,1,4.0,4.00
1,1.0,78,20,3.0,3.00
2,1.0,78,32,3.5,3.50
3,1.0,78,150,4.0,4.00
4,1.0,78,165,3.0,3.00
...,...,...,...,...,...
25745,-0.5,362,104879,4.0,-2.00
25746,-0.5,362,115569,4.0,-2.00
25747,-0.5,362,161127,4.0,-2.00
25748,-0.5,362,165101,3.5,-1.75


In [307]:
#Applies a sum to the topUsers after grouping it up by userId
tempTopUsersRating = topUsersRating.groupby('movieId').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
tempTopUsersRating.head()

,sum_similarityIndex,sum_weightedRating
movieId,,
1,14.617100,53.839748
2,9.264027,30.729283
3,3.666072,10.504818
5,1.032098,1.131183
6,8.140608,32.692144


In [308]:
#Creates an empty dataframe
Colabrecommendation_df = pd.DataFrame()
#Now we take the weighted average
Colabrecommendation_df['Colab score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
Colabrecommendation_df['movieId'] = tempTopUsersRating.index
Colabrecommendation_df.head()

,Colab score,movieId
movieId,,
1,3.683340,1
2,3.317054,2
3,2.865415,3
5,1.096004,5
6,4.015934,6


In [309]:
import numpy as np
Colabrecommendation_df = Colabrecommendation_df[~Colabrecommendation_df['Colab score'].isin([np.inf, -np.inf])]
Colabrecommendation_df.head()

,Colab score,movieId
movieId,,
1,3.683340,1
2,3.317054,2
3,2.865415,3
5,1.096004,5
6,4.015934,6


In [310]:
Colabrecommendation_df = Colabrecommendation_df.sort_values(by='Colab score', ascending=False)
Colabrecommendation_df.head(10)

,Colab score,movieId
movieId,,
96821,138.676247,96821
67734,92.130943,67734
61132,50.080224,61132
3980,32.187859,3980
44972,17.322575,44972
65682,16.655653,65682
64983,12.540377,64983
5283,11.150091,5283
1970,10.590805,1970


In [311]:
Colabrecommendation_df = Colabrecommendation_df.reset_index(drop=True)
Colabrecommendation_df

,Colab score,movieId
0,138.676247,96821
1,92.130943,67734
2,50.080224,61132
3,32.187859,3980
4,17.322575,44972
...,...,...
6676,-5.749494,2416
6677,-10.918558,1945
6678,-11.004820,45081
6679,-91.317539,63992


In [312]:
##temp
movies_df.loc[movies_df['movieId'].isin(Colabrecommendation_df.head(10)['movieId'].tolist())]

,movieId,title,genres,year
1447,1970,"Nightmare on Elm Street 3: Dream Warriors, A","[Horror, Thriller]",1987
2969,3980,Men of Honor,[Drama],2000
3779,5283,National Lampoon's Van Wilder,[Comedy],2002
6190,44972,Scary Movie 4,"[Comedy, Horror]",2006
6822,61132,Tropic Thunder,"[Action, Adventure, Comedy, War]",2008
6930,64983,Valkyrie,"[Drama, Thriller, War]",2008
6956,65682,Underworld: Rise of the Lycans,"[Action, Fantasy, Horror, Thriller]",2009
7002,67734,Adventureland,"[Comedy, Drama]",2009
7769,91535,"Bourne Legacy, The","[Action, Adventure, Drama, Thriller, IMAX]",2012
7988,96821,"Perks of Being a Wallflower, The","[Drama, Romance]",2012


# Recommendation ranked by Exclusively Collaborative method score:
showing the name and the data of movies sorted by Colab Score

In [318]:

Recommendation_df2=Recommendation_df.merge(Colabrecommendation_df,right_on="movieId",left_on="movieId" )
Recommendation_df2=Recommendation_df2.sort_values(by="Colab score",ascending=False)
Recommendation_df2.head(10)

,movieId,CB Score,title,genres,year,Colab score
4910,96821,0.111111,"Perks of Being a Wallflower, The","[Drama, Romance]",2012,138.676247
2806,67734,0.190476,Adventureland,"[Comedy, Drama]",2009,92.130943
1930,61132,0.246032,Tropic Thunder,"[Action, Adventure, Comedy, War]",2008,50.080224
4973,3980,0.111111,Men of Honor,[Drama],2000,32.187859
5087,44972,0.079365,Scary Movie 4,"[Comedy, Horror]",2006,17.322575
1275,65682,0.357143,Underworld: Rise of the Lycans,"[Action, Fantasy, Horror, Thriller]",2009,16.655653
1285,64983,0.357143,Valkyrie,"[Drama, Thriller, War]",2008,12.540377
5985,5283,0.079365,National Lampoon's Van Wilder,[Comedy],2002,11.150091
2152,1970,0.246032,"Nightmare on Elm Street 3: Dream Warriors, A","[Horror, Thriller]",1987,10.590805
267,91535,0.523810,"Bourne Legacy, The","[Action, Adventure, Drama, Thriller, IMAX]",2012,10.462144


# Defining a Hybrid Score
This score is simply derived by multiplication of the scores from two methods

The Idea is we get suggestions which had high score in both methods

In [314]:
Recommendation_df2["Hybrid Score"]=Recommendation_df2["CB Score"]*Recommendation_df2["Colab score"]
Recommendation_df2


,movieId,CB Score,title,genres,year,Colab score,Hybrid Score
0,79132,0.865079,Inception,"[Action, Crime, Drama, Mystery, Sci-Fi, Thrill...",2010,4.134447,3.576625
1,198,0.865079,Strange Days,"[Action, Crime, Drama, Mystery, Sci-Fi, Thriller]",1995,3.000000,2.595238
2,26701,0.865079,Patlabor: The Movie (Kidô keisatsu patorebâ: T...,"[Action, Animation, Crime, Drama, Film-Noir, M...",1989,3.000000,2.595238
3,6902,0.841270,Interstate 60,"[Adventure, Comedy, Drama, Fantasy, Mystery, S...",2002,4.000000,3.365079
4,85414,0.817460,Source Code,"[Action, Drama, Mystery, Sci-Fi, Thriller]",2011,3.668639,2.998967
...,...,...,...,...,...,...,...
6676,2459,0.000000,"Texas Chainsaw Massacre, The",[Horror],1974,5.000000,0.000000
6677,4217,0.000000,4 Little Girls,[Documentary],1997,4.000000,0.000000
6678,75947,0.000000,Mondo Cane,[Documentary],1962,2.000000,0.000000
6679,2401,0.000000,Pale Rider,[Western],1985,4.000000,0.000000


# Recommended Movies By Hybrid Score:
Finally by sorting the table with Hybrid Score It seems that we get better results!

In [315]:
Recommendation_df2=Recommendation_df2.sort_values(by="Hybrid Score", ascending=False)
Recommendation_df2.head(40)

,movieId,CB Score,title,genres,year,Colab score,Hybrid Score
2806,67734,0.190476,Adventureland,"[Comedy, Drama]",2009,92.130943,17.548751
4910,96821,0.111111,"Perks of Being a Wallflower, The","[Drama, Romance]",2012,138.676247,15.408472
1930,61132,0.246032,Tropic Thunder,"[Action, Adventure, Comedy, War]",2008,50.080224,12.321325
1275,65682,0.357143,Underworld: Rise of the Lycans,"[Action, Fantasy, Horror, Thriller]",2009,16.655653,5.948448
267,91535,0.523810,"Bourne Legacy, The","[Action, Adventure, Drama, Thriller, IMAX]",2012,10.462144,5.480171
100,52281,0.634921,Grindhouse,"[Action, Crime, Horror, Sci-Fi, Thriller]",2007,7.215143,4.581043
1285,64983,0.357143,Valkyrie,"[Drama, Thriller, War]",2008,12.540377,4.478706
535,3505,0.476190,No Way Out,"[Drama, Mystery, Thriller]",1987,9.208238,4.384875
1014,69526,0.396825,Transformers: Revenge of the Fallen,"[Action, Adventure, Sci-Fi, IMAX]",2009,10.418084,4.134160
176,64497,0.587302,"Day the Earth Stood Still, The","[Drama, Sci-Fi, Thriller, IMAX]",2008,6.776268,3.979713
